# Spectral Axes, Velocity Frames and Conventions

This notebook shows how to change spectral axes to represent different rest frames and Doppler conventions.

You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/velocity_frames.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/velocity_frames.ipynb" download>here</a> and selecting "Save Link As".

## Loading Modules
We start by loading the modules we will use for the data reduction. 

For display purposes, we use the static (non-interactive) `matplotlib` backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on `jupyter` lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for working with the data.
from dysh.fits.gbtfitsload import GBTFITSLoad
from astropy import units as u

# These modules are only used to download the data.
from pathlib import Path
from dysh.util.download import from_url

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
url = "http://www.gb.nrao.edu/dysh/example_data/positionswitch/data/AGBT05B_047_01/AGBT05B_047_01.raw.acs/AGBT05B_047_01.raw.acs.fits"
savepath = Path.cwd() / "data"
savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
filename = from_url(url, savepath)

## Data Loading

In [ ]:
sdfits = GBTFITSLoad(filename)
sdfits.summary()

## Data Reduction

Next we fetch and calibrate the position switched data. We will use this data to show how to change rest frames and Doppler conventions.

In [ ]:
psscan = sdfits.getps(scan=51, ifnum=0, plnum=0, fdnum=0)

Create the time-averaged spectrum.

In [ ]:
ta = psscan.timeaverage()

## Changing the x-axis of a `Spectrum` Plot
Note this changes the axis of the plot but does not affect the underlying Spectrum object.


### Default Rest Frame

The default plot uses the frequency frame and Doppler convention found in the SDFITS file.
In this case, that is topocentric frame and the optical convention.

In [ ]:
ta.plot()

### Change Rest Frame
You can change the velocity frame by supplying one of the [built-in astropy coordinate frames](https://docs.astropy.org/en/stable/coordinates/index.html#built-in-frame-classes). These are specified by string name.

For example, to plot in the barycentric frame use `"icrs"`. The change here is small, a shift of 22.8 kHz. 

In [ ]:
ta.plot(vel_frame='icrs')

In addition to the `astropy` frame names, we also allow `'topo'` and `'topocentric'`.

In [ ]:
ta.plot(vel_frame='topo')

### Doppler Convention 
One can also change the Doppler convention between `radio`, `optical`, and `relativistic`.
Here we also change the x-axis to velocity units and use LSRK frame.

In [ ]:
ta.plot(vel_frame='lsrk', doppler_convention='radio', xaxis_unit='km/s')

Finally, if you plot velocity units on the x-axis with no `vel_frame` given, it will default to the 
frame decoded from the VELDEF keyword in the header if present.

In [ ]:
ta.plot(xaxis_unit="km/s")

## Changing the Spectral Axis of the `Spectrum`.
There are two ways to accomplish this.  One returns a copy of the original Spectrum with the new spectral axis; 
the other changes the spectral axis in place.

### A.   Return a copy of the spectrum using [`Spectrum.with_frame()`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.with_frame)

In [ ]:
newspec = ta.with_frame('galactocentric')
newspec.plot()
print(f"The new spectral axis frame is {newspec.velocity_frame}")

One can see that the spectral axis of the new spectrum is different.

In [ ]:
ta.spectral_axis-newspec.spectral_axis

### B.   Change the spectral axis in place using [`Spectrum.set_frame()`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.set_frame)

In [ ]:
sa = ta.spectral_axis
ta.set_frame('gcrs')
print(f"Changed spectral axis frame to  {ta.velocity_frame}")

In [ ]:
ta.plot()

Here the original the spectral axis has changed

In [ ]:
ta.spectral_axis - sa

In [ ]:
ta.target

In [ ]:
ta.observer

## Other useful functions

### Spectral Axis Conversion

Convert the spectral axis to any units, frame, and convention with [`Spectrum.velocity_axis_to`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.velocity_axis_to). dysh understands some common synonyms like 'heliocentric' for astropy's 'hcrs'.

In [ ]:
ta.velocity_axis_to(unit="pc/Myr", toframe='heliocentric', doppler_convention='radio')

### Spectral Shift

Shift a spectrum in place to a given radial velocity or redshift with [`Spectrum.shift_spectrum_to`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D.shift_spectrum_to)

In [ ]:
print(f"before shift {ta.spectral_axis}")
ta.shift_spectrum_to(radial_velocity=0*u.km/u.s)
print(f"after shift {ta.spectral_axis}")

### Spectral Axis in Wavelengths

The default is angstrom, use [Quantity.to](https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to) to convert to other units.

In [ ]:
ta.wavelength.to('cm')

### Plot in Channels

Also, you can plot the x-axis in channel units

In [ ]:
ta.plot(xaxis_unit='chan')